In [3]:
import numpy as np
import csv

In [56]:
def preprocess(file_name):
    results = []
    with open(file_name) as csvfile:
        wine_reader = csv.reader(csvfile, dialect="excel", delimiter=';')
        for row in wine_reader:
            results.append(row)
    name_features = results[0]
    #print(name_features)
    parsed_results = []
    for row in results[1:]:
        new_row = []
        for num in row:
            new_row.append(float(num))
        parsed_results.append(new_row)
    parsed_results = np.array(parsed_results)
    n_instances = parsed_results.shape[0]
    n_features = parsed_results.shape[1] - 1
    labels = parsed_results[:, n_features]
    instances = parsed_results[:, : n_features]
    return instances, labels
white_instances, white_labels = preprocess('winequality-white.csv')
red_instances, red_labels = preprocess('winequality-red.csv')
transformed_white_labels = np.array([0 if num <6 else 1 for num in white_labels])
transformed_red_labels = np.array([0 if num <6 else 1 for num in red_labels])

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression #logistic regression classifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # to normalize data (NN is very sensitive to this!)
from sklearn.model_selection import cross_val_score, GridSearchCV #BONUS

In [58]:
X_train_white, X_test_white, y_train_white, y_test_white = train_test_split(white_instances, transformed_white_labels)

In [57]:
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(red_instances, transformed_red_labels)

In [59]:
scaler = StandardScaler()
scaler.fit(X_train_white)
scaler.fit(X_test_white)
normalised_X_train_white = scaler.transform(X_train_white)
normalised_X_test_white = scaler.transform(X_test_white)

In [60]:
scaler.fit(X_train_red)
scaler.fit(X_test_red)
normalised_X_train_red = scaler.transform(X_train_red)
normalised_X_test_red = scaler.transform(X_test_red)

In [61]:
random_forest = RandomForestClassifier()
decision_tree = DecisionTreeClassifier()
logr_regr = LogisticRegression()
random_forest.fit(normalised_X_train_red, y_train_red)
decision_tree.fit(normalised_X_train_red, y_train_red)
logr_regr.fit(normalised_X_train_red, y_train_red)
def report_accuracy(moment, clf_name, clf, x_train, y_train):
    cv_error = cross_val_score(clf, x_train, y_train, scoring="accuracy", cv=4)
    print(moment + " customisation, average accuracy of " + clf_name + " : ", cv_error.mean(), " standard deviation: ", np.std(cv_error))
report_accuracy("Before", "RFC", random_forest, normalised_X_train_red, y_train_red)  
report_accuracy("Before", "DT", decision_tree, normalised_X_train_red, y_train_red) 
report_accuracy("Before", "Log", logr_regr, normalised_X_train_red, y_train_red)          

Before customisation, average accuracy of RFC :  0.780645303466  standard deviation:  0.0189621698715
Before customisation, average accuracy of DT :  0.726403219295  standard deviation:  0.0160724698941
Before customisation, average accuracy of Log :  0.737269904851  standard deviation:  0.0267064893314
